# Import Data
don't run the first cell!

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [1]:
# don't run this cell again!
from datasets import load_dataset

ds = load_dataset("abisee/cnn_dailymail", "1.0.0")

# Data Cleaning


In [2]:
import pandas as pd

# splitting the imported data into separate pandas dataframes
print(ds.keys())
ds_train_df = ds['train'].to_pandas()
ds_validation_df = ds['validation'].to_pandas()
ds_test_df = ds['test'].to_pandas()

dict_keys(['train', 'validation', 'test'])


In [3]:
# viewing the first few observations of the dataset
print(ds_train_df.head())

                                             article  \
0  LONDON, England (Reuters) -- Harry Potter star...   
1  Editor's note: In our Behind the Scenes series...   
2  MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...   
3  WASHINGTON (CNN) -- Doctors removed five small...   
4  (CNN)  -- The National Football League has ind...   

                                          highlights  \
0  Harry Potter star Daniel Radcliffe gets £20M f...   
1  Mentally ill inmates in Miami are housed on th...   
2  NEW: "I thought I was going to die," driver sa...   
3  Five small polyps found during procedure; "non...   
4  NEW: NFL chief, Atlanta Falcons owner critical...   

                                         id  
0  42c027e4ff9730fbb3de84c1af0d2c506e41c3e4  
1  ee8871b15c50d0db17b0179a6d2beab35065f1e9  
2  06352019a19ae31e527f37f7571c6dd7f0c5da37  
3  24521a2abb2e1f5e34e6824e0f9e56904a2b0e88  
4  7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a  


In [5]:
import string

# checking for missing values
print(ds_train_df.isna().sum())

# converting letters to lowercase
ds_train_df["article"] = ds_train_df["article"].str.lower()
ds_train_df["highlights"] = ds_train_df["highlights"].str.lower()

# stripping punctuation
ds_train_df["article"] = ds_train_df["article"].str.replace("[{}]".format(string.punctuation), "", regex = True)
ds_train_df["highlights"] = ds_train_df["highlights"].str.replace("[{}]".format(string.punctuation), "", regex = True)
print(ds_train_df.head())

article       0
highlights    0
id            0
dtype: int64
                                             article  \
0  london england reuters  harry potter star dani...   
1  editors note in our behind the scenes series c...   
2  minneapolis minnesota cnn  drivers who were on...   
3  washington cnn  doctors removed five small pol...   
4  cnn   the national football league has indefin...   

                                          highlights  \
0  harry potter star daniel radcliffe gets £20m f...   
1  mentally ill inmates in miami are housed on th...   
2  new i thought i was going to die driver says  ...   
3  five small polyps found during procedure none ...   
4  new nfl chief atlanta falcons owner critical o...   

                                         id  
0  42c027e4ff9730fbb3de84c1af0d2c506e41c3e4  
1  ee8871b15c50d0db17b0179a6d2beab35065f1e9  
2  06352019a19ae31e527f37f7571c6dd7f0c5da37  
3  24521a2abb2e1f5e34e6824e0f9e56904a2b0e88  
4  7fe70cc8b12fab2d0a258fababf7d9c6

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# EDA
